In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__valeurs_de_reference WHERE categorie_1='nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

cdparametresiseeaux,categorie_1,categorie_2,categorie_3,limite_qualite,limite_qualite_unite,valeur_sanitaire_1,valeur_sanitaire_1_unite,valeur_sanitaire_2,valeur_sanitaire_2_unite,nb_rows
NO2,nitrate,None,None,0.5,mg/L,None,None,None,None,1
NO3,nitrate,None,None,50.0,mg/L,None,None,None,None,1
NO3_NO2,nitrate,None,None,1.0,None,None,None,None,None,1


In [3]:
%%sql
SELECT
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO3_NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no3_no2,
    MAX(
        CASE
            WHEN
                cdparametresiseeaux = 'NO2'
                THEN limite_qualite
        END
    ) AS limite_qualite_no2
FROM
    int__valeurs_de_reference
WHERE
    categorie_1 = 'nitrate'

Running query in 'duckdb:///../../database/data.duckdb'

limite_qualite_no3,limite_qualite_no3_no2,limite_qualite_no2
50.0,1.0,0.5


**Les règles choisies**
    
_Si on a les 3 paramètres disponibles_
- Si Nitrates < 50 mg/L et nitrites < 0,5 mg/L et nitrate/50 + nitrites/3 < 1 mg/L  --> CONFORME
- Si Nitrates >= 50 mg/L et/ou nitrites >= 0,5 mg/L et/ou nitrate/50 + nitrites/3 >= 1 mg/L --> NON CONFORME


_Si on n'a pas  nitrate/50 + nitrites/3 mais qu'on a nitrate et  nitrites_<br>
==> on le calcule et on applique comme ci-dessus

- Si on n'a pas nitrate -->  donnee_manquante
- Si on n'a pas nitrites et que  nitrate >= 50 mg/L --> NON CONFORME
- Si on n'a pas nitrites et que  nitrate < 50 mg/L --> donnee_manquant 


In [4]:
%%sql --save int__resultats_nitrites_udi_dernier
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
       int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
    SELECT
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
    FROM
        int__valeurs_de_reference
    WHERE
        categorie_1 = 'nitrate'
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT
        split_nitrites.*,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no3_no2,
        valeur_ref.limite_qualite_no2,
        split_nitrites.valtraduite_no3 / 50
        + split_nitrites.valtraduite_no2 / 3 AS valtraduite_no3_no2_calc
    FROM split_nitrites
    CROSS JOIN valeur_ref

)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
    CASE
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            < split_nitrites_with_ref.limite_qualite_no3
            AND split_nitrites_with_ref.valtraduite_no2
            < split_nitrites_with_ref.limite_qualite_no2
            AND COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                split_nitrites_with_ref.valtraduite_no3_no2_calc
            )
            < split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3
            >= split_nitrites_with_ref.limite_qualite_no3
            OR split_nitrites_with_ref.valtraduite_no2
            >= split_nitrites_with_ref.limite_qualite_no2
            OR COALESCE(
                split_nitrites_with_ref.valtraduite_no3_no2,
                split_nitrites_with_ref.valtraduite_no3_no2_calc
            )
            >= split_nitrites_with_ref.limite_qualite_no3_no2
            THEN 'non_conforme'
        WHEN
            split_nitrites_with_ref.valtraduite_no3 IS NULL
            OR (
                split_nitrites_with_ref.valtraduite_no2 IS NULL
                AND split_nitrites_with_ref.valtraduite_no3
                < split_nitrites_with_ref.limite_qualite_no3
            )
            THEN 'donnee_manquante'
        ELSE 'error'
    END AS resultat
FROM
    split_nitrites_with_ref


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
038000406,nitrate,dernier_prel,2024-11-04 10:30:00,3,conforme
038000579,nitrate,dernier_prel,2024-10-08 10:45:00,3,conforme
038000634,nitrate,dernier_prel,2025-01-30 09:20:00,3,conforme
038000869,nitrate,dernier_prel,2024-10-29 08:45:00,3,conforme
038000919,nitrate,dernier_prel,2025-02-27 09:48:00,3,conforme
038000992,nitrate,dernier_prel,2025-01-02 10:35:00,3,conforme
038001014,nitrate,dernier_prel,2025-01-15 10:10:00,3,conforme
038001205,nitrate,dernier_prel,2024-11-14 09:35:00,3,conforme
038001237,nitrate,dernier_prel,2024-06-25 10:05:00,3,conforme
038001241,nitrate,dernier_prel,2024-12-19 09:30:00,3,conforme


In [5]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, COUNT(*)
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,count_star()
non_conforme,269
conforme,20741
donnee_manquante,1149


In [6]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    * 
FROM 
    int__resultats_nitrites_udi_dernier
WHERE 
    resultat='error'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat


In [7]:
%%sql --with int__resultats_nitrites_udi_dernier
SELECT 
    resultat, MIN( cdreseau) min_cdreseau, MAX(cdreseau) AS max_cdreseau
FROM 
    int__resultats_nitrites_udi_dernier
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min_cdreseau,max_cdreseau
conforme,001000003,976003489
donnee_manquante,001000251,976003881
non_conforme,002000060,095000346


# Check 

In [8]:
%%sql
WITH
last_pvl AS (
    SELECT
        cdreseau,
        categorie,
        cdparametresiseeaux,
        limite_qualite,
        datetimeprel,
        valtraduite,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
       int__resultats_udi_communes
    WHERE
        categorie = 'nitrate'
        AND
        -- On garde les prélèvements de moins d'un an
        CURRENT_DATE - datetimeprel < INTERVAL 1 YEAR
),

valeur_ref AS (
SELECT 
    MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO3_NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no3_no2,
        MAX(
            CASE
                WHEN
                    cdparametresiseeaux = 'NO2'
                    THEN limite_qualite
            END
        ) AS limite_qualite_no2
FROM int__valeurs_de_reference
WHERE categorie_1='nitrate'    
),

split_nitrites AS (
    SELECT
        last_pvl.cdreseau,
        last_pvl.categorie,
        COUNT(DISTINCT last_pvl.cdparametresiseeaux) AS nb_parametres,
        MAX(last_pvl.datetimeprel) AS dernier_prel_datetime,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n03,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_n02,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.datetimeprel
            END
        ) AS dernier_prel_datetime_no3_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no2,
        MAX(
            CASE
                WHEN
                    last_pvl.cdparametresiseeaux = 'NO3_NO2'
                    THEN last_pvl.valtraduite
            END
        ) AS valtraduite_no3_no2
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
    GROUP BY
        last_pvl.cdreseau,
        last_pvl.categorie
),

split_nitrites_with_ref AS (
    SELECT 
        split_nitrites.*,
        valeur_ref.limite_qualite_no3,
        valeur_ref.limite_qualite_no3_no2,
        valeur_ref.limite_qualite_no2,
        split_nitrites.valtraduite_no3 / 50
        + split_nitrites.valtraduite_no2 / 3 AS valtraduite_no3_no2_calc
    FROM split_nitrites
    CROSS JOIN valeur_ref

)

SELECT
    split_nitrites_with_ref.cdreseau,
    split_nitrites_with_ref.categorie,
    'dernier_prel' AS periode,
    split_nitrites_with_ref.dernier_prel_datetime,
    split_nitrites_with_ref.nb_parametres,
  split_nitrites_with_ref.valtraduite_no2,
  split_nitrites_with_ref.valtraduite_no3,
  split_nitrites_with_ref.valtraduite_no3_no2,
  split_nitrites_with_ref.valtraduite_no3_no2_calc
FROM
    split_nitrites_with_ref  
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293')
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,valtraduite_no2,valtraduite_no3,valtraduite_no3_no2,valtraduite_no3_no2_calc
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,0.000,14.000,0.280,0.28
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,0.000,2.400,0.050,0.048
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,0.000,6.000,0.120,0.12
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,0.000,3.700,0.070,0.07400000000000001
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,3,0.000,59.300,0.000,1.186
073002059,nitrate,dernier_prel,2024-10-23 09:28:00,1,0.000,None,None,None
088002296,nitrate,dernier_prel,2024-09-27 12:00:00,1,0.000,None,None,None
095000346,nitrate,dernier_prel,2025-02-27 08:35:00,3,0.000,47.000,1.000,0.94
973000028,nitrate,dernier_prel,2024-12-02 11:30:00,2,0.000,0.000,None,0.0


In [9]:
%%sql --with int__resultats_nitrites_udi_dernier 
 SELECT 
   *
 FROM
   int__resultats_nitrites_udi_dernier
WHERE
    cdreseau IN( '002000060','095000346','001000003','976003554','088002296','073002059','973000028','001000260','001000268','001000293',)
ORDER BY 
    cdreseau

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,periode,dernier_prel_datetime,nb_parametres,resultat
001000003,nitrate,dernier_prel,2025-01-21 12:35:00,3,conforme
001000260,nitrate,dernier_prel,2024-12-06 12:02:00,3,conforme
001000268,nitrate,dernier_prel,2025-01-13 09:57:00,3,conforme
001000293,nitrate,dernier_prel,2025-01-28 11:29:00,3,conforme
002000060,nitrate,dernier_prel,2024-10-18 08:50:00,3,non_conforme
073002059,nitrate,dernier_prel,2024-10-23 09:28:00,1,donnee_manquante
088002296,nitrate,dernier_prel,2024-09-27 12:00:00,1,donnee_manquante
095000346,nitrate,dernier_prel,2025-02-27 08:35:00,3,non_conforme
973000028,nitrate,dernier_prel,2024-12-02 11:30:00,2,conforme
